In [1]:
from __future__ import division

In [2]:
import tensorflow as tf
import tflearn

In [3]:
import numpy as np

In [4]:
sess = tf.Session()
# sess = tf_debug.LocalCLIDebugWrapperSession(sess)
lr = tf.placeholder(tf.float32, shape=[])
optim = tf.train.AdamOptimizer(learning_rate=lr)
# sgd_optim = tf.train.GradientDescentOptimizer(learning_rate=lr)

In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
data_size = mnist.train.num_examples

In [7]:
from tqdm import tqdm

In [8]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [9]:
# import BayesianNN

In [10]:
# bnn = BayesianNN.BayesianFC([4,32,32,1])

In [11]:
import Bayesian_Conv_Layer, Bayesian_DeConv_Layer, Bayesian_FC_Layer, Bayesian_Net

In [12]:
def bbnet(i, o, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_FC_Layer.Bayesian_FC(i, o, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [13]:
def bbnet_conv(i, f, stride, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_Conv_Layer.Bayesian_Conv(i, f, filter_stride=stride, 
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [14]:
def bbnet_deconv(ih, iw, ic, f, fh, fw, stride, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.relu):
    return Bayesian_DeConv_Layer.Bayesian_DeConv((ih, iw), ic, f, filter_stride=stride, 
                                         filter_height=fh,
                                         filter_width=fw,
                                         qw_mean_initial=mean_init, 
                                         qw_std_initial=std_init, 
                                         qb_std_initial=std_init,
                                         pw_mean=prior_mean,
                                         pw_sigma=prior_std,
                                         pb_mean=prior_mean,
                                         pb_sigma=prior_std,
                                         activation=activation
                                         )

In [15]:
std_init = 0.1
mean_init = 0.0
prior_mean = 0.0
prior_std = 1.0
l1 = bbnet_conv(1, 8, 2, mean_init, std_init, prior_mean, prior_std)
l2 = bbnet_conv(8, 8, 2, mean_init, std_init, prior_mean, prior_std)
l3 = bbnet(288, 144, mean_init, std_init, prior_mean, prior_std)
l4 = bbnet_deconv(12,12,1, 8,4,4, 2, mean_init, std_init, prior_mean, prior_std)
l5 = bbnet_deconv(26,26,8, 1,3,3, 1, mean_init, std_init, prior_mean, prior_std, activation=tf.nn.sigmoid)

# l1 = Bayesian_FC_Layer.Bayesian_FC(4, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l2 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l3 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l4 = Bayesian_FC_Layer.Bayesian_FC(32, 32, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)
# l5 = Bayesian_FC_Layer.Bayesian_FC(32, 1, activation=tf.identity, qw_mean_initial=mean_init, qw_std_initial=std_init, qb_std_initial=std_init)

In [16]:
from Bayesian_Net import log_gaussian_pdf

In [17]:
def bnet_prob(pred, target):
    return log_gaussian_pdf(pred, target, 0.1)

In [18]:
bnet = Bayesian_Net.Bayesian_Net([l1, l2, l3, l4, l5], bnet_prob)

In [19]:
batch_size = 32

In [20]:
epochs = 1000

In [21]:
bnn_data_input = tf.placeholder(tf.float32, shape=[None, 28*28*1])
bnn_data_target = tf.placeholder(tf.float32, shape=[None, 28*28*1])
kls = tf.placeholder(tf.float32, shape=[])

def bnet_model(local_reparam_trick=False):
    input_tensor = tf.reshape(bnn_data_input, shape=[-1,28,28,1])
    
    net = l1.sample(input_tensor, local_reparam_trick)
    net = l2.sample(net, local_reparam_trick)
    
    size = tf.reduce_prod(tf.shape(net)[1:])
    # Flatten
    net = tf.reshape(net, shape=[-1, size])
    
    net = l3.sample(net, local_reparam_trick)
    
    # Unflatten
    net = tf.reshape(net, shape=[-1,12,12,1])
    
    net = l4.sample(net, local_reparam_trick)
    net_out = l5.sample(net, local_reparam_trick)
    
    #Flatten again
    net_out = tf.reshape(net_out, shape=[-1, 28*28*1])
    
    return net_out

bnet_loss, klloss, dataloss = bnet.loss(bnet_model, bnn_data_target,kl_scaling=kls, N=8)
bnet_output = bnet_model(local_reparam_trick=False)

copy_old = bnet.copy_variational_parameters()
kl_div = bnet.kl_new_and_old()

sq_loss = tf.square(bnet_output-bnn_data_target)
accuracy = tf.reduce_mean(sq_loss)

grads_to_apply = optim.compute_gradients(bnet_loss)
# print(grads_to_apply)

clipped_policy_grads = []
for (grad, var) in grads_to_apply:
    if grad is not None:
        clipped_policy_grads.append((tf.clip_by_norm(grad, 10), var))

# clipped_policy_grads = grads_to_apply
grad_op = optim.apply_gradients(clipped_policy_grads)

sess.run(tf.global_variables_initializer())

In [22]:
bnn_lr = 0.001

In [23]:
epoch_loss=[]
kl_loss=[]
data_loss=[]
kl_divs=[]
accs=[]

In [24]:
def filter_digits(x,y,allowed):
    new_x = []
    new_y = []
    for nx,ny in zip(x,y):
        if np.argmax(ny)<=allowed:
            new_x.append(nx)
            new_y.append(ny)
    return new_x, new_y

In [38]:
for e in tqdm(range(epochs)):
    
    x, y = mnist.train.next_batch(batch_size)
    
#     if e < epochs*0.5:
#         x, y = filter_digits(x,y,3)
    
    kl = batch_size/data_size  
    
    feed_dict={kls:kl, bnn_data_input: x, bnn_data_target:x, lr:bnn_lr}
    
    _, l, kloss, dloss = sess.run([grad_op, bnet_loss, klloss, dataloss], feed_dict=feed_dict)

    epoch_loss.append(l)
    
    if e % 50 == 0 and e > 0:
        div = sess.run(kl_div)
        sess.run(copy_old)
        kl_divs.append(div)
    
    if e % 5 == 0:
        epoch_loss.append(l) 
        kl_loss.append(kloss)
        data_loss.append(dloss)
        
    if e % 100 == 0:
        test_dict = {bnn_data_input: mnist.test.images, bnn_data_target: mnist.test.images}
        acc = sess.run([accuracy], feed_dict=test_dict)
        accs.append(acc)

100%|██████████| 1000/1000 [02:20<00:00,  7.71it/s]


In [39]:
loss_p = figure(width=500, height=500)

# Loss
# loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=0.5, color="green")

loss_p.line(range(len(kl_loss)), kl_loss, line_width=0.5, color="orange")

loss_p.line(range(len(data_loss)), data_loss, line_width=0.5, color="purple")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=1, color="green")

show(loss_p)

In [40]:
loss_kl = figure(width=500, height=500)

loss_kl.line(range(len(kl_divs)), kl_divs, line_width=0.5, color="red")

show(loss_kl)

In [41]:
acc_g = figure(width=500, height=500)

acc_g.line(range(len(accs)), accs, line_width=0.5, color="blue")

show(acc_g)

In [42]:
test_image = mnist.validation.images[7]

In [43]:
img = np.reshape(test_image, [28,28])
# img = ascent()/256.0

plot = figure(plot_width=img.shape[0]*10, plot_height=img.shape[1]*10, 
                 x_range=[0, img.shape[0]], y_range=[0, img.shape[1]])
plot.image(image=[np.flipud(img)], x=[0], y=[0],
           dw=[img.shape[0]], dh=[img.shape[1]])
show(plot)

In [44]:
predicted_image = sess.run(bnet_output, feed_dict={bnn_data_input: [test_image]})

In [45]:
img = np.reshape(predicted_image, [28,28])
# img = ascent()/256.0

plot = figure(plot_width=img.shape[0]*10, plot_height=img.shape[1]*10, 
                 x_range=[0, img.shape[0]], y_range=[0, img.shape[1]])
plot.image(image=[np.flipud(img)], x=[0], y=[0],
           dw=[img.shape[0]], dh=[img.shape[1]])
show(plot)

In [46]:
preds = sess.run(bnet_output, feed_dict={bnn_data_input: [test_image]})
for _ in range(100-1):
    p_i = sess.run(bnet_output, feed_dict={bnn_data_input: [test_image]})
    preds = np.concatenate([preds, p_i])

In [47]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 0, preds)

In [48]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 0, preds)

In [49]:
img = np.reshape(pred_means, [28,28])
# img = ascent()/256.0

plot = figure(plot_width=img.shape[0]*10, plot_height=img.shape[1]*10, 
                 x_range=[0, img.shape[0]], y_range=[0, img.shape[1]])
plot.image(image=[np.flipud(img)], x=[0], y=[0],
           dw=[img.shape[0]], dh=[img.shape[1]])
show(plot)

In [50]:
img = np.reshape(pred_vars, [28,28])

plot = figure(plot_width=28*10, plot_height=28*10, 
                 x_range=[0, 28], y_range=[0, 28])
plot.image(image=[np.flipud(img)], x=[0], y=[0],
           dw=[28], dh=[28])
show(plot)